# Hyperparameters optimization


`````{margin}
````{dropdown} Necessary imports
```{code-block} python
from dataclasses import asdict

import networkx as nx
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from myst_nb import glue
from scipy.stats import beta

from colosseum import config
from colosseum.agent.agents.episodic import QLearningEpisodic
from colosseum.agent.agents.infinite_horizon import QLearningContinuous
from colosseum.benchmark.run import instantiate_and_get_exp_instances_from_agents_and_benchmarks_for_hyperopt
from colosseum.experiment.experiment_instances import run_experiment_instances
from colosseum.hyperopt import DEFAULT_HYPEROPT_CONF, DEFAULT_HYPEROPT_CONF_NONTABULAR, SMALL_HYPEROPT_CONF, \
    SMALL_HYPEROPT_CONF_NONTABULAR, HyperOptConfig
from colosseum.hyperopt.selection import get_optimal_agents_configs
from colosseum.hyperopt.utils import sample_agent_configs_and_benchmarks_for_hyperopt
from colosseum.mdp.custom_mdp import CustomEpisodic
from colosseum.mdp.river_swim import RiverSwimEpisodic
from colosseum.mdp.simple_grid import SimpleGridContinuous

experiments_folder = "tutorial"
experiment_name = "bench_run"

config.set_experiments_folder(experiments_folder, experiment_name)
config.set_hyperopt_folder(experiments_folder, experiment_name)

config.enable_verbose_logging()

seed = 42
```
````
`````

In [1]:
import shutil
from dataclasses import asdict

import networkx as nx
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from myst_nb import glue
from scipy.stats import beta

from colosseum import config
from colosseum.agent.agents.episodic import QLearningEpisodic
from colosseum.agent.agents.infinite_horizon import QLearningContinuous
from colosseum.benchmark.run import instantiate_and_get_exp_instances_from_agents_and_benchmarks_for_hyperopt
from colosseum.experiment.experiment_instances import run_experiment_instances
from colosseum.hyperopt import DEFAULT_HYPEROPT_CONF, DEFAULT_HYPEROPT_CONF_NONTABULAR, SMALL_HYPEROPT_CONF, \
    SMALL_HYPEROPT_CONF_NONTABULAR, HyperOptConfig
from colosseum.hyperopt.selection import retrieve_best_agent_config_from_hp_folder
from colosseum.hyperopt.utils import sample_agent_configs_and_benchmarks_for_hyperopt
from colosseum.mdp.custom_mdp import CustomEpisodic
from colosseum.mdp.river_swim import RiverSwimEpisodic
from colosseum.mdp.simple_grid import SimpleGridContinuous


def pretty_print(hpc):
  index = [
      "Default tabular",
      "Small tabular",
      "Default non-tabular",
      "Small non-tabular",
  ]
  columns = list(asdict(DEFAULT_HYPEROPT_CONF))
  data = [
      list(map(str, asdict(DEFAULT_HYPEROPT_CONF).values())),
      list(map(str, asdict(SMALL_HYPEROPT_CONF).values())),
      list(
          map(
              lambda x: x.__name__ if "emission_maps" in str(x) else str(x),
              asdict(DEFAULT_HYPEROPT_CONF_NONTABULAR).values(),
          )
      ),
      list(
          map(
              lambda x: x.__name__ if "emission_maps" in str(x) else str(x),
              asdict(SMALL_HYPEROPT_CONF_NONTABULAR).values(),
          )
      ),
  ]
  
  return pd.DataFrame(data, index, columns).T
  
experiments_folder = "tutorial"
experiment_name = "hyperopt"

config.set_hyperopt_folder(experiments_folder, experiment_name)

2022-10-15 16:43:54.313750: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 16:43:54.396451: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2022-10-15 16:43:54.801693: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-15 16:43:54.801731: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-10-15 16:43:54.801735: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


As the [_Agent implementation_ tutorial](../tutorials/agent-implementation.md) explains, agent classes must define a sampling space for its hyperparameters.
These sampling spaces are used to conduct a random search optimization procedure with the objective of minimizing the cumulative regret across a set of randomly sampled environments.
The random sampling procedure for environments is defined for each {{col}} MDP class, and aims to provide a varied set of MDPs of up to moderate scale.


## Hyperparameters optimization configurations

The [`HyperOptConfig`](../pdoc_files/colosseum/hyperopt/config.html#HyperOptConfig) class controls the parameters of the hyperparameter optimization procedure.
There are four default hyperparameters optimization configurations available in the package, which are reported in the table below.
[`DEFAULT_HYPEROPT_CONF`](../pdoc_files/colosseum/hyperopt/config.html#DEFAULT_HYPEROPT_CONF) is the default hyperparameters optimization configuration for tabular agents,
[`SMALL_HYPEROPT_CONF`](../pdoc_files/colosseum/hyperopt/config.html#SMALL_HYPEROPT_CONF) is a quick hyperparameters optimization configuration for tabular agents that can be used for quick testing,
[`DEFAULT_HYPEROPT_CONF_NONTABULAR`](../pdoc_files/colosseum/hyperopt/config.html#DEFAULT_HYPEROPT_CONF_NONTABULAR) is the default hyperparameters optimization configuration for non-tabular agents, and
[`SMALL_HYPEROPT_CONF_NONTABULAR`](../pdoc_files/colosseum/hyperopt/config.html#SMALL_HYPEROPT_CONF_NONTABULAR) is the default hyperparameters optimization configuration for non-tabular agents that can be used for quick testing.

In [2]:
pretty_print(DEFAULT_HYPEROPT_CONF).style.set_table_attributes('style="font-size: 14px"')

,Default tabular,Small tabular,Default non-tabular,Small non-tabular
seed,42,42,42,42
n_timesteps,250000,30000,250000,50000
max_interaction_time_s,300,120,600,60
n_samples_agents,50,2,50,2
n_samples_mdps,5,2,5,2
log_every,100000,10000,50000,10000
emission_map,None,None,StateInfo,StateInfo
mdp_classes,None,None,None,None
n_seeds,3,1,3,1


## Hyperparameters optimization

Running the hyperparameters optimization procedure is very similar to running a benchmark.
The only difference is that the benchmarks are automatically sampled.

In [3]:
# Define a custom small scale hyperparameters optimization procedure
hpoc = HyperOptConfig(
    seed=42,
    n_timesteps=20_000,
    max_interaction_time_s=40,
    n_samples_agents=1,
    n_samples_mdps=1,
    log_every=500,
    n_seeds=1,
)

# Take the q-learning agents as running example
agent_cls = [QLearningContinuous, QLearningEpisodic]

# Create the benchmarks for the given agents classes and hyperparameters optimzation configuration
hyperopt_agents_and_benchmarks = sample_agent_configs_and_benchmarks_for_hyperopt(agent_cls, hpoc)

# Obtain the instances and run them locally
hp_exp_instances = instantiate_and_get_exp_instances_from_agents_and_benchmarks_for_hyperopt(
    hyperopt_agents_and_benchmarks
)
run_experiment_instances(hp_exp_instances)

# Compute the best hyperparameters, which, by default, minimize the average normalized cumulative regret
optimal_agent_configs = retrieve_best_agent_config_from_hp_folder()

In [4]:
print(optimal_agent_configs[QLearningEpisodic])
print()
print(optimal_agent_configs[QLearningContinuous])

prms_0/QLearningEpisodic.p=0.05
prms_0/QLearningEpisodic.UCB_type="bernstein"
prms_0/QLearningEpisodic.c_1 = 0.4126
prms_0/QLearningEpisodic.c_2 = 1.0458
prms_0/QLearningEpisodic.min_at = 0.1467


prms_0/QLearningContinuous.h_weight = 0.4126
prms_0/QLearningContinuous.span_approx_weight = 1.0458
prms_0/QLearningContinuous.min_at = 0.1467



In [5]:
shutil.rmtree(config.get_hyperopt_folder())